# Announcement

The video differs slightly from the notebook. In particular, I have updated the ngrok code a bit.

I am also counting on you using the video from Chapter 14 to understand the html portion of the server - I do not go over that again.

The web page you will see in the video is an older, less modern version I have used in past. As you know from Chapter 14, I tried to modernize it a bit.

<center>
<h1>Flask Web Server For Colab</h1>
</center>

<hr>

The simplest approach to your final project is to use this notebook as the foundation for your web server. I have tried to set it up so the main components work on any dataset. That leaves you with two main tasks:

1. Loading in new files that go with your new dataset. I have the notebook set up to work with the titanic files. You will have to change over to your new files.

2. Reworking the html so you have an interface that fits the new features in your dataset. Your goal is to take info from a user, make predictions, then feed those predictions back to the user. The current html is focused on the Titanic. But you saw how to change it in Chapter 14.

# I. Before you even get to this notebook

This notebook assumes you have 11 files defined and ready to load in:

1. You have 4 threshold tables as csv files on GitHub.

2. Your have 3 joblib files for knn, logreg and lgb on GitHub.

3. You have an ANN file (`.keras`), on GitHub.

4. You have your Lime explainer on GitHub.

5. You have a fitted pipeline on GitHub.

6. You have a markdown file on GitHub that explains the design decisions you made in building your pipeline.

In essence, it assumes you have done all your wrangling and model tuning prior. And saved your results to github. You are then ready to tackle this notebook.

# II. My general strategy for wrangling in this notebook

1. You will get a new row from the user. This will contain non-transformed values.

2. You will build a test set of exactly one row, the row you just got. So a DataFrame of exactly one row.

3. You will call the pre-fitted pipeline (you read in from GitHub) on this test set with the `transform` method.

4. This will give you a new table (of only one row) with transformated values. Convert that table to numpy matrix.

You are now ready to feed this to machine learning models.

# III. Moving to a real web server

The easiest I can see is to use railway.com. pythonanywhere.com is also an option. Either allows you to set up a flask server and gives you local storage.

The only reason I am mentioning these options is in case you want to show off your server to someone, e.g., a perspective employer. It might be good to have it hosted some place where he or she can try it out easily without you having to crank up this notebook.

## Getting your own authtoken

You will need to create a (free) account on ngrok. Then from the dashboard, select tab on left as a shown below. Copy the authtoken and replace mine with yours (code at end of this notebook).

<img src='https://www.dropbox.com/scl/fi/32jymtl6q2n5elumrt1wl/Screenshot-2025-01-30-at-4.45.34-PM.png?rlkey=ud0k66atjhjsn2vwblivue9bw&raw=1'>

## You need to do this!

If you use my token instead of yours it will screw me up, as well as you and possibly your classmates. A free token only allows one session to run at a time. So if you are using my token, I cannot. Or I am using my token, you cannot.

So get your own token!

## I could have moved these imports into my library

If you did that, you won't need to import as below.

In [1]:
from joblib import load
from flask import Flask
from flask import request
import os
import tensorflow as tf

## Bring in your own library

In [2]:
github_name = 'marvnc'
repo_name = 'cs523'
source_file = 'library.py'
url = f'https://raw.githubusercontent.com/{github_name}/{repo_name}/main/{source_file}'
!rm $source_file
!wget $url
%run -i $source_file

--2025-06-06 07:43:50--  https://raw.githubusercontent.com/marvnc/cs523/main/library.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49866 (49K) [text/plain]
Saving to: ‘library.py’

library.py          100%[===================>]  48.70K  --.-KB/s    in 0.01s   

2025-06-06 07:43:50 (3.65 MB/s) - ‘library.py’ saved [49866/49866]



## Load the files

You will need to replace what I have (Titanic-related files and urls) with your own.


## Set-up pipeline documentation

Assumes markdown description on GitHub. Will load it then convert it to html. Can later insert into `fpage`.

In [3]:
# url to Pipeline documentation in markup format.
pipe_doc_url = 'https://raw.githubusercontent.com/MarvNC/cs523/refs/heads/main/s25_final_pipeline_documentation.md'
!rm 's25_final_pipeline_documentation.md'
!wget $pipe_doc_url
with open('s25_final_pipeline_documentation.md', 'r', encoding='utf-8') as file:
    pipe_md_content = file.read()

--2025-06-06 07:43:50--  https://raw.githubusercontent.com/MarvNC/cs523/refs/heads/main/s25_final_pipeline_documentation.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7205 (7.0K) [text/plain]
Saving to: ‘s25_final_pipeline_documentation.md’

s25_final_pipeline_ 100%[===================>]   7.04K  --.-KB/s    in 0s      

2025-06-06 07:43:50 (14.4 MB/s) - ‘s25_final_pipeline_documentation.md’ saved [7205/7205]



In [4]:
pipe_md_content

'# Credit Card Approvals Data Pipeline Documentation\n\n## Pipeline Overview\n\nThis pipeline preprocesses the Credit Card Approvals dataset, focusing on numerical features to prepare them for machine learning modeling. It applies outlier detection and treatment, followed by feature scaling. Categorical features like \'Gender\', \'PriorDefault\', \'Employed\', and \'DriversLicense\' are assumed to be already in a suitable binary (0/1) format and are not processed by this specific pipeline. Similarly, this pipeline does not perform missing value imputation, as indicated by the dataset\'s state at this stage.\n\n```python\napprovals_transformer = Pipeline(steps=[\n    # Gender: already categorical 0 or 1\n    # Age: numerical, so we might transform to normalize it then apply tukey for outliers\n    (\'tukey_age\', CustomTukeyTransformer(target_column=\'Age\', fence=\'outer\')),\n    (\'scale_age\', CustomRobustTransformer(target_column=\'Age\')),\n    # Debt: numerical, so normalize and 

## Bring in models

Comes from Tuning notebook. You will need to load the 4 models and 4 threshold tables.

In [5]:
# assumes you have things stored in a single repo
model_path = 'MarvNC/cs523/main/s25_'

In [6]:
# load trained lgb model
full_path = f'https://raw.githubusercontent.com/{model_path}final_lgb_model.joblib'
!rm 's25_final_lgb_model.joblib'
!wget $full_path
lgb_model = load('s25_final_lgb_model.joblib')


--2025-06-06 07:43:51--  https://raw.githubusercontent.com/MarvNC/cs523/main/s25_final_lgb_model.joblib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439924 (430K) [application/octet-stream]
Saving to: ‘s25_final_lgb_model.joblib’

s25_final_lgb_model 100%[===================>] 429.61K  --.-KB/s    in 0.05s   

2025-06-06 07:43:51 (8.98 MB/s) - ‘s25_final_lgb_model.joblib’ saved [439924/439924]



In [7]:
# load trained logistic regression model
full_path = f'https://raw.githubusercontent.com/{model_path}final_logreg_model.joblib'
!rm 's25_final_logreg_model.joblib'
!wget $full_path
logreg_model = load('s25_final_logreg_model.joblib')


--2025-06-06 07:43:57--  https://raw.githubusercontent.com/MarvNC/cs523/main/s25_final_logreg_model.joblib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4636 (4.5K) [application/octet-stream]
Saving to: ‘s25_final_logreg_model.joblib’

s25_final_logreg_mo 100%[===================>]   4.53K  --.-KB/s    in 0s      

2025-06-06 07:43:57 (46.8 MB/s) - ‘s25_final_logreg_model.joblib’ saved [4636/4636]



In [8]:
## load trained knn model
full_path = f'https://raw.githubusercontent.com/{model_path}final_knn_model.joblib'
!rm 's25_final_knn_model.joblib'
!wget $full_path
knn_model = load('s25_final_knn_model.joblib')


--2025-06-06 07:43:57--  https://raw.githubusercontent.com/MarvNC/cs523/main/s25_final_knn_model.joblib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 334534 (327K) [application/octet-stream]
Saving to: ‘s25_final_knn_model.joblib’

s25_final_knn_model 100%[===================>] 326.69K  --.-KB/s    in 0.04s   

2025-06-06 07:43:57 (7.33 MB/s) - ‘s25_final_knn_model.joblib’ saved [334534/334534]



In [9]:
## load trained ann model
full_path = f'https://raw.githubusercontent.com/{model_path}final_ann_model.keras'
!rm 's25_final_ann_model.keras'
!wget $full_path
ann_model = tf.keras.models.load_model('s25_final_ann_model.keras', custom_objects={'LeakyReLU': tf.keras.layers.LeakyReLU()})

--2025-06-06 07:43:57--  https://raw.githubusercontent.com/MarvNC/cs523/main/s25_final_ann_model.keras
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44433 (43K) [application/octet-stream]
Saving to: ‘s25_final_ann_model.keras’

s25_final_ann_model 100%[===================>]  43.39K  --.-KB/s    in 0.01s   

2025-06-06 07:43:58 (3.27 MB/s) - ‘s25_final_ann_model.keras’ saved [44433/44433]



In [10]:
ann_model.input_shape  #should match feature columns you will get user info for

(None, 9)

## Bring in threshold tables

Comes from Tuning notebook.

In [11]:
# now threshold tables - comes from tuning notebook. Change in final project.

logreg_thresholds = pd.read_csv(f'https://raw.githubusercontent.com/{model_path}final_logreg_thresholds.csv').round(2)
knn_thresholds = pd.read_csv(f'https://raw.githubusercontent.com/{model_path}final_knn_thresholds.csv').round(2)
lgb_thresholds = pd.read_csv(f'https://raw.githubusercontent.com/{model_path}final_lgb_thresholds.csv').round(2)
ann_thresholds = pd.read_csv(f'https://raw.githubusercontent.com/{model_path}final_ann_thresholds.csv').round(2)

## Build threshold tables as HTML

Once we have HTML version, can insert that in page.



In [12]:
lgb_table = lgb_thresholds.to_html(index=False, justify='center').replace('<td>','<td style="text-align: center;">')
logreg_table = logreg_thresholds.to_html(index=False, justify='center').replace('<td>','<td style="text-align: center;">')
knn_table = knn_thresholds.to_html(index=False, justify='center').replace('<td>', '<td style="text-align: center;">')
ann_table = ann_thresholds.to_html(index=False, justify='center').replace('<td>', '<td style="text-align: center;">')

## Now fitted transformer

Comes from Wrangling notebook.

In [13]:
!pip install dill
import dill as pickle

In [14]:
full_path = f'https://raw.githubusercontent.com/{model_path}final_fully_fitted_pipeline.pkl'
!rm 's25_final_fully_fitted_pipeline.pkl'
!wget $full_path
fitted_transformer = load('s25_final_fully_fitted_pipeline.pkl')


--2025-06-06 07:44:01--  https://raw.githubusercontent.com/MarvNC/cs523/main/s25_final_fully_fitted_pipeline.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1327 (1.3K) [application/octet-stream]
Saving to: ‘s25_final_fully_fitted_pipeline.pkl’

s25_final_fully_fit 100%[===================>]   1.30K  --.-KB/s    in 0s      

2025-06-06 07:44:01 (79.2 MB/s) - ‘s25_final_fully_fitted_pipeline.pkl’ saved [1327/1327]



# Now Lime

Comes from Tuning notebook.

In [15]:
# feature_names  = [...]  #change to match your own dataset
feature_names = [
    'Gender',
    'Age',
    'Debt',
    'YearsEmployed',
    'PriorDefault',
    'Employed',
    'CreditScore',
    'DriversLicense',
    'Income',
]

In [16]:
%%capture
!pip install lime

In [17]:
import lime
from lime import lime_tabular


In [18]:
# get trained lime explainer
full_path = f'https://raw.githubusercontent.com/{model_path}final_lime_explainer.pkl'
!rm 's25_final_lime_explainer.pkl'
!wget $full_path
with open('s25_final_lime_explainer.pkl', 'rb') as file:
    lime_explainer = pickle.load(file)

--2025-06-06 07:44:09--  https://raw.githubusercontent.com/MarvNC/cs523/main/s25_final_lime_explainer.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13172 (13K) [application/octet-stream]
Saving to: ‘s25_final_lime_explainer.pkl’

s25_final_lime_expl 100%[===================>]  12.86K  --.-KB/s    in 0.001s  

2025-06-06 07:44:09 (17.1 MB/s) - ‘s25_final_lime_explainer.pkl’ saved [13172/13172]



## Done loading from GitHub

At this point, you have all your hard-earned work loaded in the server and ready to use.

The final step is tailoring the web page and functions that deal with it to fit your specific dataset.

# Break into pieces to make it a little more understandable

I'll show you the basic template page below. You will notice placeholders with form `%...%.` I will use Python functions to compute what gets inserted into these places. So this single template will grow quite large as we fill in pieces. Again, you do not have to worry about that, unless you want to.

Here is basic template.

In [19]:
# Main page template using a two-column layout
fpage_template = '''
<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>Credit Approval Prediction</title>
%enhanced_styling%
</head>

<body>
  <div class="container">
    <div class="left-column">
      <!-- Header -->
      <div class="page-header">
        <h1>Credit Approval Prediction</h1>
        <img src='https://s.abcnews.com/images/Business/GTY_credit_score_jt_140226_16x9_992.jpg' height=200>
      </div>

      <!-- Prediction Form -->
      <div class="form-panel">
        <h2>Enter Applicant Information</h2>
        %form_section%
      </div>

      <!-- Results Section -->
      <div class="results-panel">
        %results_section%
      </div>
    </div>

    <div class="right-column" id="tableContainer">
      <!-- This space is intentionally left empty for threshold tables -->
      %threshold_tables%
    </div>
  </div>

  <script>
    /**
     * Improved toggle function with scrolling
     *
     * This function:
     * 1. Hides all tables
     * 2. If the clicked table was hidden, shows it and scrolls to it
     * 3. If the clicked table was already visible, keeps it hidden
     *
     * @param {string} tableId - The ID of the table to toggle
     */
  function toggleTable(tableId) {
    // Get the table element
    var table = document.getElementById(tableId);

    // Get all tables
    var tables = document.querySelectorAll('.table-wrapper');

    // Check if this table is currently visible (using computed style)
    var computedStyle = window.getComputedStyle(table);
    var isVisible = computedStyle.display !== 'none';

    // First, hide all tables
    tables.forEach(function(t) {
      t.style.display = 'none';
    });

    // If the table was not visible, show it and scroll to it
    if (!isVisible) {
      table.style.display = 'block';

      // Scroll the right column into view
      document.getElementById('tableContainer').scrollIntoView({
        behavior: 'smooth',
        block: 'start'
      });
    }
  // If it was visible, it will remain hidden (toggled off)
}
  </script>
</body>
</html>
'''

# TO-DO for you

You will be working with your own dataset. Change the titles and find an image that goes with your dataset and change the template above.


## Set-up styling

This is a complicated function that defines styling to give the page a more professional look. I have left comments. I do not expect you to change it although you can if you want.

In [20]:
import markdown

def get_enhanced_styling():
    """
    Returns enhanced CSS styling for the Titanic prediction page.

    This function contains all the CSS styling rules for the page, organized by section.
    Each section has detailed comments explaining what the styles control and how they work.

    Returns:
    --------
    str
        A string containing all CSS styling rules wrapped in <style> tags
    """
    return '''
    <style>
      /*********************************************
       * BASE STYLES
       * These styles apply to the entire document
       *********************************************/

      /*
       * Basic body styling
       * - Sets background color to light blue-gray
       * - Sets default text color to dark gray
       * - Uses modern, sans-serif font stack
       * - Sets comfortable line spacing
       * - Removes default margins and padding
       */
      body {
        background-color: #f0f4f8;  /* Light blue-gray background */
        color: #333;                /* Dark gray text */
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;  /* Modern font stack */
        line-height: 1.6;           /* Comfortable line spacing for readability */
        margin: 0;                  /* Remove default margin */
        padding: 0;                 /* Remove default padding */
      }

      /*********************************************
       * LAYOUT STRUCTURE
       * Controls the two-column layout of the page
       *********************************************/

      /*
       * Main container
       * - Creates a flex container for the two-column layout
       * - Ensures the container takes up at least the full viewport height
       */
      .container {
        display: flex;              /* Use flexbox for layout */
        min-height: 100vh;          /* Ensure container is at least full viewport height */
      }

      /*
       * Left column
       * - Contains the form and results
       * - Takes up 45% of the container width
       * - Has padding around all sides
       */
      .left-column {
        width: 45%;                 /* Set column width to 45% of container */
        padding: 20px;              /* Add spacing around content */
        box-sizing: border-box;     /* Include padding in width calculation */
      }

      /*
       * Right column
       * - Reserved for threshold tables
       * - Takes up 55% of the container width
       * - Has padding around all sides
       * - Position is relative for child positioning
       */
      .right-column {
        width: 55%;                 /* Set column width to 55% of container */
        padding: 20px;              /* Add spacing around content */
        box-sizing: border-box;     /* Include padding in width calculation */
        position: relative;         /* For positioning tables */
      }

      /*********************************************
       * HEADER SECTION
       * Styles for the page header and title
       *********************************************/

      /*
       * Page header container
       * - Has a blue gradient background
       * - White text
       * - Rounded corners
       * - Bottom margin to separate from content below
       * - Subtle shadow for depth
       */
      .page-header {
        background: linear-gradient(135deg, #23283a 0%, #2b5876 100%);  /* Gradient background */
        color: white;               /* White text */
        padding: 20px;              /* Internal spacing */
        border-radius: 8px;         /* Rounded corners */
        margin-bottom: 25px;        /* Space below header */
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
      }

      /*
       * Main page heading (h1)
       * - No margin to avoid extra space
       * - White text color
       * - Larger font size
       * - Medium font weight
       */
      .page-header h1 {
        margin: 0;                  /* Remove default margin */
        color: white;               /* White text */
        font-size: 32px;            /* Larger text */
        font-weight: 500;           /* Medium weight */
      }

      /*
       * Header image styling
       * - Responsive width
       * - Maintain aspect ratio
       * - Rounded corners
       * - Space above image
       */
      .page-header img {
        max-width: 100%;            /* Responsive width */
        height: auto;               /* Maintain aspect ratio */
        border-radius: 4px;         /* Rounded corners */
        margin-top: 15px;           /* Space above image */
      }

      /*********************************************
       * PANEL STYLING
       * Styles for the form and results panels
       *********************************************/

      /*
       * Form panel styling
       * - White background
       * - Rounded corners
       * - Internal padding
       * - Subtle shadow
       * - Bottom margin for spacing
       */
      .form-panel {
        background-color: white;    /* White background */
        border-radius: 8px;         /* Rounded corners */
        padding: 25px;              /* Internal spacing */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
        margin-bottom: 25px;        /* Space below panel */
      }

      /*
       * Form panel heading
       * - No top margin to avoid extra space
       * - Blue text color
       * - Bottom border for visual separation
       * - Padding below text for spacing from border
       * - Medium font weight
       */
      .form-panel h2 {
        margin-top: 0;              /* Remove top margin */
        color: #2b5876;             /* Blue heading text */
        border-bottom: 2px solid #f0f4f8;  /* Light bottom border */
        padding-bottom: 10px;       /* Space between text and border */
        font-weight: 500;           /* Medium weight */
      }

      /*
       * Results panel styling
       * - White background
       * - Rounded corners
       * - Internal padding
       * - Subtle shadow
       */
      .results-panel {
        background-color: white;    /* White background */
        border-radius: 8px;         /* Rounded corners */
        padding: 25px;              /* Internal spacing */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
      }

      /*
       * Results panel heading
       * - No top margin to avoid extra space
       * - Blue text color
       * - Medium font weight
       */
      .results-panel h2 {
        margin-top: 0;              /* Remove top margin */
        color: #2b5876;             /* Blue heading text */
        font-weight: 500;           /* Medium weight */
      }

      /*********************************************
       * FORM CONTROL STYLING
       * Styles for form inputs, dropdowns, and buttons
       *********************************************/

      /*
       * Text input styling
       * - Full width of container
       * - Internal padding
       * - Light border
       * - Rounded corners
       * - Box sizing to include padding in width
       * - Comfortable font size
       * - Top margin for spacing from label
       */
      input[type="text"] {
        width: 100%;                /* Full width */
        padding: 10px 15px;         /* Vertical and horizontal padding */
        border: 1px solid #ddd;     /* Light gray border */
        border-radius: 4px;         /* Rounded corners */
        box-sizing: border-box;     /* Include padding in width */
        font-size: 16px;            /* Comfortable font size */
        margin-top: 6px;            /* Space from label above */
      }

      /*
       * Dropdown select styling
       * - Full width of container
       * - Internal padding
       * - Light border
       * - Rounded corners
       * - White background
       * - Box sizing to include padding in width
       * - Comfortable font size
       * - Top margin for spacing from label
       * - Remove default appearance for custom styling
       * - Add custom dropdown arrow
       */
      select {
        width: 100%;                /* Full width */
        padding: 10px 15px;         /* Vertical and horizontal padding */
        border: 1px solid #ddd;     /* Light gray border */
        border-radius: 4px;         /* Rounded corners */
        background-color: white;    /* White background */
        box-sizing: border-box;     /* Include padding in width */
        font-size: 16px;            /* Comfortable font size */
        margin-top: 6px;            /* Space from label above */
        appearance: none;           /* Remove default browser styling */

        /* Add custom dropdown arrow as background image */
        background-image: url("data:image/svg+xml;charset=US-ASCII,%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20width%3D%22292.4%22%20height%3D%22292.4%22%3E%3Cpath%20fill%3D%22%23007CB2%22%20d%3D%22M287%2069.4a17.6%2017.6%200%200%200-13-5.4H18.4c-5%200-9.3%201.8-12.9%205.4A17.6%2017.6%200%200%200%200%2082.2c0%205%201.8%209.3%205.4%2012.9l128%20127.9c3.6%203.6%207.8%205.4%2012.8%205.4s9.2-1.8%2012.8-5.4L287%2095c3.5-3.5%205.4-7.8%205.4-12.8%200-5-1.9-9.2-5.5-12.8z%22%2F%3E%3C%2Fsvg%3E");
        background-repeat: no-repeat;
        background-position: right 10px center;  /* Position arrow on right side */
        background-size: 12px;      /* Size of arrow */
      }

      /*
       * Form group container
       * - Adds bottom margin for spacing between form elements
       */
      .form-group {
        margin-bottom: 20px;        /* Space below group */
      }

      /*
       * Form label styling
       * - Display as block for layout
       * - Bottom margin for spacing
       * - Medium font weight
       * - Blue text color for consistency
       */
      label {
        display: block;             /* Make labels block elements */
        margin-bottom: 6px;         /* Space below label */
        font-weight: 500;           /* Medium weight */
        color: #2b5876;             /* Blue text */
      }

      /*
       * Submit button styling
       * - Blue/purple gradient background
       * - White text
       * - No border
       * - Comfortable padding
       * - Larger font size
       * - Rounded corners
       * - Pointer cursor on hover
       * - Subtle shadow for depth
       * - Smooth transition for hover effects
       */
      .submit-btn {
        background: linear-gradient(135deg, #2b5876 0%, #4e4376 100%);  /* Gradient background */
        color: white;               /* White text */
        border: none;               /* No border */
        padding: 12px 24px;         /* Vertical and horizontal padding */
        font-size: 18px;            /* Larger text */
        border-radius: 4px;         /* Rounded corners */
        cursor: pointer;            /* Hand cursor on hover */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.2);  /* Subtle shadow */
        transition: all 0.3s ease;  /* Smooth transition for hover effects */
      }

      /*
       * Submit button hover state
       * - Larger shadow for "lifting" effect
       * - Slight upward movement for interaction feedback
       */
      .submit-btn:hover {
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3);  /* Larger shadow */
        transform: translateY(-2px);  /* Move up slightly */
      }

      /*********************************************
       * RESULTS BUTTONS STYLING
       * Styles for the model result buttons
       *********************************************/

      /*
       * Models list container
       * - Uses flexbox for layout
       * - Allows wrapping to multiple lines if needed
       * - Adds spacing between buttons
       * - Bottom margin for spacing
       */
      .models-list {
        display: flex;              /* Use flexbox for layout */
        flex-wrap: wrap;            /* Allow wrapping to multiple lines */
        gap: 10px;                  /* Spacing between buttons */
        margin-bottom: 20px;        /* Space below button group */
      }

      /*
       * Model button styling
       * - Blue/purple gradient background
       * - White text
       * - No border
       * - Comfortable padding
       * - Rounded corners
       * - Appropriate font size
       * - Pointer cursor on hover
       * - Subtle shadow for depth
       * - Smooth transition for hover effects
       * - Only take up needed space (not full width)
       * - Center text
       */
      .model-btn {
        background: linear-gradient(135deg, #2b5876 0%, #4e4376 100%);  /* Gradient background */
        color: white;               /* White text */
        border: none;               /* No border */
        padding: 10px 16px;         /* Vertical and horizontal padding */
        border-radius: 4px;         /* Rounded corners */
        font-size: 16px;            /* Comfortable font size */
        cursor: pointer;            /* Hand cursor on hover */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
        transition: all 0.3s ease;  /* Smooth transition for hover effects */
        flex: 0 0 auto;             /* Only take up needed space */
        text-align: center;         /* Center text */
      }

      /*
       * Model button hover state
       * - Larger shadow for "lifting" effect
       * - Slight upward movement for interaction feedback
       */
      .model-btn:hover {
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);  /* Larger shadow */
        transform: translateY(-2px);  /* Move up slightly */
      }

      /*
       * Documentation button styling
       * - Blue gradient background (different from model buttons)
       * - White text
       * - No border
       * - Appropriate padding
       * - Rounded corners
       * - Comfortable font size
       * - Pointer cursor on hover
       * - Left margin for spacing
       * - Subtle shadow for depth
       * - Smooth transition for hover effects
       */
      .doc-button {
        background: linear-gradient(135deg, #3a7bd5 0%, #00d2ff 100%);  /* Blue gradient */
        color: white;               /* White text */
        border: none;               /* No border */
        padding: 8px 15px;          /* Vertical and horizontal padding */
        border-radius: 4px;         /* Rounded corners */
        font-size: 16px;            /* Comfortable font size */
        cursor: pointer;            /* Hand cursor on hover */
        margin-left: 10px;          /* Space to the left */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
        transition: all 0.3s ease;  /* Smooth transition for hover effects */
      }

      /*
       * Documentation button hover state
       * - Larger shadow for "lifting" effect
       * - Slight upward movement for interaction feedback
       */
      .doc-button:hover {
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);  /* Larger shadow */
        transform: translateY(-2px);  /* Move up slightly */
      }

      /*********************************************
       * MODAL STYLING
       * Styles for the documentation popup modal
       *********************************************/

      /*
       * Modal container
       * - Hidden by default
       * - Fixed position covering the entire viewport
       * - High z-index to appear above other content
       * - Semi-transparent black background overlay
       * - Scrollable if content is too tall
       */
      .modal {
        display: none;              /* Hidden by default */
        position: fixed;            /* Fixed position */
        z-index: 1000;              /* High z-index to be on top */
        left: 0;                    /* Align to left edge */
        top: 0;                     /* Align to top edge */
        width: 100%;                /* Full width */
        height: 100%;               /* Full height */
        background-color: rgba(0,0,0,0.5);  /* Semi-transparent background */
        overflow: auto;             /* Allow scrolling if needed */
      }

      /*
       * Modal content container
       * - White background
       * - Centered with margin
       * - Internal padding
       * - Rounded corners
       * - Limited width and height
       * - Scrollable if content is too tall
       * - Relative position for close button
       * - Shadow for depth
       */
      .modal-content {
        background-color: #fefefe;  /* White background */
        margin: 2% auto;            /* Center horizontally with space at top */
        padding: 20px;              /* Internal spacing */
        border-radius: 8px;         /* Rounded corners */
        width: 80%;                 /* Width relative to viewport */
        max-width: 900px;           /* Maximum width */
        max-height: 90vh;           /* Maximum height relative to viewport */
        overflow-y: auto;           /* Vertical scrolling if needed */
        position: relative;         /* For positioning close button */
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);  /* Shadow for depth */
      }

      /*
       * Close button styling
       * - Absolute position in top right corner
       * - White text on blue background
       * - Bold text
       * - Pointer cursor on hover
       * - Comfortable padding
       * - Rounded corners
       * - Shadow for depth
       * - Smooth transition for hover effects
       */
      .close-button {
        position: absolute;         /* Absolute position */
        top: 15px;                  /* Top spacing */
        right: 20px;                /* Right spacing */
        color: #fff;                /* White text */
        background-color: #2b5876;  /* Blue background */
        font-size: 16px;            /* Comfortable font size */
        font-weight: bold;          /* Bold text */
        cursor: pointer;            /* Hand cursor on hover */
        padding: 5px 15px;          /* Vertical and horizontal padding */
        border-radius: 4px;         /* Rounded corners */
        box-shadow: 0 2px 4px rgba(0,0,0,0.2);  /* Shadow for depth */
        transition: all 0.3s ease;  /* Smooth transition for hover effects */
      }

      /*
       * Close button hover state
       * - Darker background color
       * - Larger shadow for "lifting" effect
       */
      .close-button:hover {
        background-color: #23283a;  /* Darker blue on hover */
        box-shadow: 0 4px 8px rgba(0,0,0,0.3);  /* Larger shadow */
      }

      /*********************************************
       * TABLE STYLING
       * Styles for the threshold tables
       *********************************************/

      /*
       * Table wrapper container
       * - Hidden by default
       * - Sticky position to stay visible when scrolling
       * - Top spacing when sticky
       */
      .table-wrapper {
        display: none;              /* Hidden by default */
        position: sticky;           /* Sticky positioning */
        top: 20px;                  /* Top spacing when sticky */
      }

      /*
       * Common styling for both table columns
       * - White background
       * - Rounded corners
       * - Subtle shadow
       * - Internal padding
       * - Bottom margin for spacing
       */
      .table1, .table2 {
        background-color: white;    /* White background */
        border-radius: 8px;         /* Rounded corners */
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);  /* Subtle shadow */
        padding: 15px;              /* Internal spacing */
        margin-bottom: 20px;        /* Space below */
      }

      /*
       * Left table column
       * - Float left
       * - 45% width of container
       */
      .table1 {
        float: left;                /* Float left */
        width: 45%;                 /* 45% width */
      }

      /*
       * Right table column
       * - Float right
       * - 45% width of container
       * - Left margin for spacing from left column
       */
      .table2 {
        float: right;               /* Float right */
        width: 45%;                 /* 45% width */
        margin-left: 10px;          /* Space from left column */
      }

      /*
       * Ensemble result styling
       * - Larger font size
       * - Medium font weight
       * - Blue text color
       * - Top margin for spacing
       * - Top padding for spacing
       * - Top border for visual separation
       * - Clear both floats to appear below buttons
       */
      .ensemble-result {
        font-size: 24px;            /* Larger text */
        font-weight: 500;           /* Medium weight */
        color: #2b5876;             /* Blue text */
        margin-top: 15px;           /* Space above */
        padding: 10px;              /* Padding */
        border-top: 2px solid #f0f4f8;  /* Light top border */
        clear: both;                /* Clear floats */
      }

      /*********************************************
       * MARKDOWN CONTENT STYLING
       * Styles for the rendered markdown in the documentation modal
       *********************************************/

      /*
       * Markdown container styling
       * - Modern font stack
       * - Comfortable line height
       */
      #markdown-content {
        font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Arial, sans-serif;
        line-height: 1.6;           /* Comfortable line height */
      }

      /*
       * Markdown headings
       * - Blue text color
       */
      #markdown-content h1, #markdown-content h2 {
        color: #2c3e50;             /* Blue text */
      }

      /*
       * Markdown h2 headings
       * - Bottom border for visual separation
       * - Bottom padding for spacing from border
       */
      #markdown-content h2 {
        border-bottom: 1px solid #eee;  /* Light bottom border */
        padding-bottom: 5px;        /* Space between text and border */
      }

      /*
       * Markdown code blocks
       * - Light gray background
       * - Padding for spacing
       * - Rounded corners
       * - Monospace font
       */
      #markdown-content code {
        background-color: #f5f5f5;  /* Light gray background */
        padding: 2px 4px;           /* Vertical and horizontal padding */
        border-radius: 3px;         /* Rounded corners */
        font-family: monospace;     /* Monospace font */
      }

      /*
       * Markdown lists
       * - Left padding for indentation
       */
      #markdown-content ul, #markdown-content ol {
        padding-left: 25px;         /* Left padding */
      }

      /*
       * Markdown paragraphs
       * - Bottom margin for spacing
       */
      #markdown-content p {
        margin-bottom: 16px;        /* Space below */
      }

      /*
       * Markdown blockquotes
       * - Left and right padding
       * - Gray text color
       * - Left border for visual distinction
       * - Bottom margin for spacing
       */
      #markdown-content blockquote {
        padding: 0 1em;             /* Horizontal padding */
        color: #6a737d;             /* Gray text */
        border-left: 0.25em solid #dfe2e5;  /* Left border */
        margin: 0 0 16px 0;         /* Bottom margin */
      }

      /*
       * Markdown tables
       * - Collapse borders
       * - Full width
       * - Bottom margin for spacing
       */
      #markdown-content table {
        border-collapse: collapse;  /* Collapse borders */
        width: 100%;                /* Full width */
        margin-bottom: 16px;        /* Space below */
      }

      /*
       * Markdown table cells
       * - Padding for spacing
       * - Light border
       */
      #markdown-content table th, #markdown-content table td {
        padding: 6px 13px;          /* Vertical and horizontal padding */
        border: 1px solid #dfe2e5;  /* Light border */
      }

      /*
       * Markdown table alternating rows
       * - Light gray background for even rows
       */
      #markdown-content table tr:nth-child(2n) {
        background-color: #f6f8fa;  /* Light gray background */
      }

      /*********************************************
       * RESPONSIVE DESIGN
       * Adjustments for smaller screens
       *********************************************/

      /*
       * Media query for screens smaller than 1200px
       * - Changes layout from two columns to one column
       * - Makes both columns full width
       * - Changes right column positioning
       */
      @media (max-width: 1200px) {
        .container {
          flex-direction: column;   /* Stack columns vertically */
        }

        .left-column, .right-column {
          width: 100%;              /* Full width */
        }

        .right-column {
          position: static;         /* Normal positioning */
        }
      }
    </style>
    '''


## These are helper functions

Again, you should not have to change them although you are welcome to do so.

In [21]:

def text_input(name, label, placeholder=""):
    """
    Generate HTML for a styled text input field with label.

    This function creates a form group containing a label and text input field.
    The styling is handled by CSS classes defined in get_enhanced_styling().

    Parameters:
    -----------
    name : str
        The input field name (used for form submission and label connection)
    label : str
        The descriptive text for the label
    placeholder : str, optional
        Placeholder text shown in the input when empty

    Returns:
    --------
    str
        HTML markup for the label and input field wrapped in a form group
    """
    return f'''
    <div class="form-group">
      <label for="{name}">{label}</label>
      <input type="text" id="{name}" name="{name}_field" placeholder="{placeholder}">
    </div>
    '''

def dropdown_select(name, label, options):
    """
    Generate HTML for a styled dropdown select with label.

    This function creates a form group containing a label and select dropdown.
    The styling is handled by CSS classes defined in get_enhanced_styling().

    Parameters:
    -----------
    name : str
        The select field name (used for form submission and label connection)
    label : str
        The descriptive text for the label
    options : dict
        Dictionary of {value: display_text} pairs for the dropdown options

    Returns:
    --------
    str
        HTML markup for the label and select dropdown wrapped in a form group
    """
    options_html = ""
    for value, text in options.items():
        options_html += f'<option value="{value}">{text}</option>\n'

    return f'''
    <div class="form-group">
      <label for="{name}">{label}</label>
      <select id="{name}" name="{name}">
        {options_html}
      </select>
    </div>
    '''

def submit_button(label="Submit"):
    """
    Generate HTML for a styled submit button.

    This function creates a submit button for a form.
    The styling is handled by CSS classes defined in get_enhanced_styling().

    Parameters:
    -----------
    label : str, optional
        The text displayed on the button (default: "Submit")

    Returns:
    --------
    str
        HTML markup for the submit button
    """
    return f'''
    <button type="submit" class="submit-btn">{label}</button>
    '''


In [22]:
def get_results_section():
    """
    Generate HTML for the results section with buttons.

    This function creates the results section of the page, including:
    - A heading
    - The pipeline documentation button (inserted via placeholder)
    - A row data display area (inserted via placeholder)
    - Buttons for each model's results
    - An ensemble result display

    The styling is handled by CSS classes defined in get_enhanced_styling().

    Returns:
    --------
    str
        HTML markup for the complete results section with placeholders
    """
    return '''
    <h2>Results</h2>
    %pipeline_docs%
    <h3>%row_data%</h3>

    <div class="models-list">
      <button class="model-btn" onclick="toggleTable('lgb')">
        LGB: %lgb%
      </button>

      <button class="model-btn" onclick="toggleTable('knn')">
        KNN: %knn%
      </button>

      <button class="model-btn" onclick="toggleTable('logreg')">
        LogReg: %logreg%
      </button>

      <button class="model-btn" onclick="toggleTable('ann')">
        ANN: %ann%
      </button>
    </div>

    <div class="ensemble-result">
      Ensemble: %ensemble%
    </div>
    '''

def get_threshold_tables_section():
    """
    Generate HTML for the threshold tables section.

    This function creates the table containers for all model results.
    Each model has two tables: a threshold table and a LIME explanation table.
    By default, all tables are hidden and will only be shown when their
    corresponding button is clicked.

    The styling is handled by CSS classes defined in get_enhanced_styling().

    Returns:
    --------
    str
        HTML markup for all the model result tables with placeholders
    """
    return '''
    <!-- LGB Model Tables -->
    <div id="lgb" class='table-wrapper'>
      <div class="table1">
        <center><h2>LGB Threshold Table</h2></center>
        %lgb_table%
      </div>
      <div class="table2">
      <center><h2>LGB Lime Explanation</h2></center>
        %lgb_lime_table%
      </div>
    </div>

    <!-- KNN Model Tables -->
    <div id="knn" class='table-wrapper'>
      <div class="table1">
      <center><h2>KNN Threshold Table</h2></center>
        %knn_table%
      </div>
      <div class="table2">
      <center><h2>KNN Lime Explanation</h2></center>
        %knn_lime_table%
      </div>
    </div>

    <!-- LogReg Model Tables -->
    <div id="logreg" class='table-wrapper'>
      <div class="table1">
      <center><h2>Logistic Regression Threshold Table</h2></center>
        %logreg_table%
      </div>
      <div class="table2">
      <center><h2>Logistic Regression Lime Explanation</h2></center>
        %logreg_lime_table%
      </div>
    </div>

    <!-- ANN Model Tables -->
    <div id="ann" class='table-wrapper'>
      <div class="table1">
      <center><h2>ANN Threshold Table</h2></center>
        %ann_table%
      </div>
      <div class="table2">
      <center><h2>ANN Lime Explanation</h2></center>
        %ann_lime_table%
      </div>
    </div>
    '''

def get_pipeline_documentation(md_content):

    # Convert markdown to HTML using Python-Markdown
    html_content = markdown.markdown(
        md_content,
        extensions=['tables', 'fenced_code', 'nl2br']
    )

    # Create complete HTML for popup with styled content and JS functionality
    documentation_html = f"""
    <!-- Button to open modal -->
    <button class="doc-button" id="openDocBtn">See Preprocessing Steps</button>

    <!-- The Modal/Popup -->
    <div id="docModal" class="modal">
        <!-- Modal content -->
        <div class="modal-content">
            <span class="close-button" id="closeModal">CLOSE</span>
            <div id="markdown-content">
                {html_content}
            </div>
        </div>
    </div>

    <!-- JavaScript for modal functionality -->
    <script>
        // Get modal elements
        const modal = document.getElementById("docModal");
        const btn = document.getElementById("openDocBtn");
        const closeBtn = document.getElementById("closeModal");

        // Open modal when button is clicked
        btn.onclick = function() {{
            modal.style.display = "block";
        }}

        // Close modal ONLY when CLOSE is clicked
        closeBtn.onclick = function() {{
            modal.style.display = "none";
        }}

        // Prevent modal from closing when clicking on the content
        document.querySelector(".modal-content").onclick = function(event) {{
            event.stopPropagation();
        }}
    </script>
    """

    return documentation_html

pipeline_docs_html = get_pipeline_documentation(pipe_md_content)


# This actually starts gluing things together

You can see that there are a series of string `replace` methods. They start with the `fpage_template` then end up in `fpage` as full website.

In [23]:
def create_template_page(config, fpage_template, pipeline_docs_html, lgb_table, logreg_table, knn_table, ann_table):
    """
    Main function to generate the complete Titanic prediction page.

    This function:
    1. Gets all the components (styling, form, results, tables, docs)
    2. Replaces all placeholders in the template
    3. Returns the complete HTML page

    Parameters:
    -----------

    Returns:
    --------
    str
        The complete HTML for the Titanic prediction page
    """

    # Compute all the pieces
    form_html = complete_form(config)
    results_section_html = get_results_section()
    threshold_tables_html = get_threshold_tables_section()
    enhanced_styling = get_enhanced_styling()

    # Replace all placeholders
    fpage = fpage_template.replace('%enhanced_styling%', enhanced_styling)
    fpage = fpage.replace('%form_section%', form_html)
    fpage = fpage.replace('%results_section%', results_section_html)
    fpage = fpage.replace('%threshold_tables%', threshold_tables_html)
    fpage = fpage.replace('%pipeline_docs%', pipeline_docs_html)

    # These threshold tables do not change so add them now
    fpage = fpage.replace('%lgb_table%', lgb_table)
    fpage = fpage.replace('%logreg_table%', logreg_table)
    fpage = fpage.replace('%knn_table%', knn_table)
    fpage = fpage.replace('%ann_table%', ann_table)

    return fpage

# TO-DO for you

This is set-up for the Titanic. You will need to make changes here for your own dataset.

In [24]:
def get_dataset_config():
    """
    Centralized configuration for Approval dataset fields.
    This single function defines both form elements and data processing.

    Returns a dictionary where each key is a field name, and each value is
    a dictionary of properties for that field.
    """
    return {
        "Gender": {
            "form_field": "gender_field",
            "label": "Gender",
            "type": "categorical",
            "input_type": "select",
            "column_name": "Gender",
            "process": lambda x: int(x) if x in ["0", "1"] else np.nan,
            "options": {
                "unknown": "Unknown",
                "1": "Female",
                "0": "Male"
            }
        },
        "Age": {
            "form_field": "age_field",
            "label": "Age",
            "placeholder": "e.g. 35",
            "type": "numeric",
            "input_type": "text",
            "column_name": "Age",
            "process": lambda x: float(x) if float(x) >= 0 else np.nan,
        },
        "Debt": {
            "form_field": "debt_field",
            "label": "Debt",
            "placeholder": "e.g. 2.5",
            "type": "numeric",
            "input_type": "text",
            "column_name": "Debt",
            "process": lambda x: float(x) if float(x) >= 0 else np.nan,
        },
        "YearsEmployed": {
            "form_field": "years_employed_field",
            "label": "Years Employed",
            "placeholder": "e.g. 5.0",
            "type": "numeric",
            "input_type": "text",
            "column_name": "YearsEmployed",
            "process": lambda x: float(x) if float(x) >= 0 else np.nan,
        },
        "PriorDefault": {
            "form_field": "prior_default_field",
            "label": "Prior Default?",
            "type": "categorical",
            "input_type": "select",
            "column_name": "PriorDefault",
            "process": lambda x: int(x) if x in ["0", "1"] else np.nan,
            "options": {
                "unknown": "Unknown",
                "1": "Yes",
                "0": "No"
            }
        },
        "Employed": {
            "form_field": "employed_field",
            "label": "Employed?",
            "type": "categorical",
            "input_type": "select",
            "column_name": "Employed",
            "process": lambda x: int(x) if x in ["0", "1"] else np.nan,
            "options": {
                "unknown": "Unknown",
                "1": "Yes",
                "0": "No"
            }
        },
        "CreditScore": {
            "form_field": "credit_score_field",
            "label": "Credit Score",
            "placeholder": "e.g. 5",
            "type": "numeric",
            "input_type": "text",
            "column_name": "CreditScore",
            "process": lambda x: float(x) if float(x) >= 0 else np.nan,
        },
        "DriversLicense": {
            "form_field": "drivers_license_field",
            "label": "Has Driver's License?",
            "type": "categorical",
            "input_type": "select",
            "column_name": "DriversLicense",
            "process": lambda x: int(x) if x in ["0", "1"] else np.nan,
            "options": {
                "unknown": "Unknown",
                "1": "Yes",
                "0": "No"
            }
        },
        "Income": {
            "form_field": "income_field",
            "label": "Income",
            "placeholder": "e.g. 5000",
            "type": "numeric",
            "input_type": "text",
            "column_name": "Income",
            "process": lambda x: float(x) if float(x) >= 0 else np.nan,
        }
    }

# Get dataset configuration
config = get_dataset_config()


# A note about debugging

Using print statements in any of the functions we have defined will not work. They will get swallowed up by the threaded server. What I have found most useful is to put `assert False, f"info I want to print out"` in a function. This will cause an error on the web page that you can see. You can then make a change and reload the web page. Don't need to restart the server.

Note that if you change `get_dataset_config`, you will need to rerun several cells, i.e., the one above and the one below, to get a modified `fpage`.

# This function is key

It takes the config file you build and integrates it with the last piece you need, the set of questions to ask the user.

In [25]:
def complete_form(config, form_id="row_info", action="data", method="POST"):
    """
    Generate form HTML using the dataset configuration.
    """
    form_elements = []

    # Create HTML for each field based on its type
    for field_id, field_config in config.items():
        if field_config["input_type"] == "text":
            form_elements.append(
                text_input(
                    field_config["form_field"],
                    field_config["label"],
                    field_config.get("placeholder", "")
                )
            )
        elif field_config["input_type"] == "select":
            form_elements.append(
                dropdown_select(
                    field_config["form_field"],
                    field_config["label"],
                    field_config["options"]
                )
            )

    # Build the complete form
    form_html = f'''
    <form id="{form_id}" action="{action}" method="{method}">
        <input type='hidden' id='hidden1' value='hidden value'/>
        {''.join(form_elements)}
        {submit_button("Evaluate")}
    </form>
    '''

    return form_html

fpage = create_template_page(config, fpage_template, pipeline_docs_html, lgb_table, logreg_table, knn_table, ann_table)  #start the glueing process

### You now have full **static** template in fpage

The dynamic part is taking form data, creating predictions and Lime tables, and filling in pieces in template.

There are no changes needed from here on.

# Server data handling method

No need to change. Called when user clicks `Evaluate` button on frontend page.

Runs pipeline then does prediction.



In [26]:
def handle_data(columns, fitted_transformer, config, column_order):
    """
    Process form data using the dataset configuration.

    Parameters:
    -----------
    columns : dict
        Dictionary containing form field values, with field names as keys
    fitted_transformer : Pipeline
        Fitted sklearn Pipeline for transforming the input data
    config : dict
        Dataset configuration from get_dataset_config()

    Returns:
    --------
    tuple
        (transformed_row, yhat_lgb, yhat_knn, yhat_logreg, yhat_ann)
    """

    # string to print
    assert_stuff = ''

    # Create DataFrame with columns in the expected order
    row_df = pd.DataFrame(columns=column_order)
    row_df.loc[0] = np.nan  # Add blank row

    # Process form values and fill the DataFrame

    assert_stuff += f'\n columns:{columns}\n rowdf_columns: {row_df.columns}'

    for field_id, field_config in config.items():
        form_field = field_config["form_field"]
        column_name = field_config["column_name"]

        assert_stuff += f'\nform field {form_field} and col name {column_name}'

        if form_field in columns and column_name in row_df.columns:
            # Apply the field's processing function and assign to the correct column
            processed_value = field_config["process"](columns[form_field])
            assert_stuff += f'\n{processed_value}, source: {columns[form_field]}'
            row_df.loc[0, column_name] = processed_value

    # assert False, assert_stuff

    # Run pipeline
    row_transformed = fitted_transformer.transform(row_df)

    # Grab added row
    new_row = row_transformed.loc[0].to_list()
    new_row = np.array(new_row)
    new_row = np.reshape(new_row, (1,-1)) if len(new_row.shape)==1 else new_row

    # Get predictions
    yhat_lgb, yhat_knn, yhat_logreg, yhat_ann = get_prediction(new_row)

    return new_row, yhat_lgb, yhat_knn, yhat_logreg, yhat_ann

In [27]:
# Helper function to change ann output into pairs of probabilities in Lime - use as is
def ann_proba(rows):
  yhat = ann_model.predict(rows)
  result = [[1.0-p[0],p[0]] for p in yhat]  #wrangle into proba form
  x = np.array(result)
  return x

In [28]:
# Helper function to build dataframe for Lime results - use as is
def create_lime_table(the_explainer):
  the_probs = the_explainer.predict_proba.round(2)
  the_list = the_explainer.as_list()
  df = pd.DataFrame(columns=['Condition', 'Probs', "Contribution"])
  for i,row in enumerate(the_list):
    df.loc[i] = [row[0],the_probs,row[1]]
  return df

# Get predictions from 4 models

This function is called, last thing, by `handle_data`. You should not have to change it.


In [29]:
# I probably should pass all the models in but I'm using them as globals. My bad. Being lazy.

def get_prediction(row):

  global lgb_model, knn_model, logreg_model, ann_model

  assert len(row.shape)==2 and row.shape[0]==1, f'Expecting nested numpy array but got {row}'
  assert logreg_model.n_features_in_ == len(row[0]), f'length mismatch with what was trained on and row to predict: {logreg_model.n_features_in_} and {len(row[0])}'

  #lgb
  lgb_raw = lgb_model.predict_proba(row)  #predict last row, we just tacked on
  yhat_lgb = lgb_raw[:,1]

  #KNN
  knn_raw = knn_model.predict_proba(row)
  yhat_knn = knn_raw[:,1]

  #logreg
  logreg_raw = logreg_model.predict_proba(row)
  yhat_logreg = logreg_raw[:,1]


  #ANN
  yhat_ann = ann_model.predict(row)[:,0]

  return [yhat_lgb, yhat_knn, yhat_logreg, yhat_ann]

## This function called during runtime

It takes fpage and values computed from user data, and updates the page.

In [30]:
def create_page(page, **fillers):
  new_page = page[:]  #copy
  for k,v in fillers.items():
    new_page = new_page.replace(f'%{str(k)}%', str(v))
  return new_page

# IX. The actual server

I have it threaded so that it will run behind the scenes.

We will also see a url printed in the output cell. This is what we can send to anyone on the planet with a browser and they can hook up to our server. You want the url that has the form `http://....ngrok-free.app`.

If you have errors with the ngrok code below, I would consult Gemini/Claude/chatGPT, etc. I am not an expert on ngrok and they probably are.



## Important: get your own auth code

If you use mine, it limits us to one server running at a time. Hence, if you are using my auth code, I cannot run. So get your own! It is not that hard.
See Step 2 here: https://medium.com/@thexpertdev/ngrok-tutorial-for-beginners-how-to-expose-localhost-to-the-internet-and-test-webhooks-70845654fced



In [31]:
import threading
!pip install pyngrok
from pyngrok import ngrok

## This is my token

Get your own!

In [32]:
ngrok_auth_token = '2y7kHHEylcni2iIQaYi98bVUwwX_7W1uereDdmDtHnGareT6h'      #'2IHAcPIUsjsfalQCtXV7IMFPsKr_3KbLamqn7KCjEPEQL3ToS'

### If you need to restart the server

`kill_port` will bring down the server and so you can restart it. Your old user page should be deleted and you should create a new one with new link.

That said, I see no reason you should have to kill the server once it is running. You can make changes to any of the functions we have seen and server will not complain and use those changes. Cool.

In [33]:
import psutil
import subprocess

def kill_port(port):
    try:
        # Get the process ID using lsof
        output = subprocess.check_output(["lsof", "-t", "-i", f":{port}"])
        pid = int(output.decode().strip())

        # Kill the process
        process = psutil.Process(pid)
        process.terminate()  # Gracefully terminate first
        print(f"Process on port {port} (PID: {pid}) has been terminated.")

    except subprocess.CalledProcessError:
        print(f"No process found running on port {port}")
    except psutil.NoSuchProcess:
        print(f"Process already terminated")
    except Exception as e:
        print(f"Error occurred: {e}")


In [34]:
!killall ngrok >/dev/null
kill_port(5000)

ngrok: no process found
No process found running on port 5000


In [35]:
os.environ['FLASK_DEBUG'] = "development"
app = Flask(__name__)
port = 5000
#Setting an auth token allows us to open a tunnel
ngrok.set_auth_token(ngrok_auth_token)
# Open a ngrok tunnel to the HTTP server
connection = ngrok.connect(port)
print(f'Connection: {connection}')
public_url = connection if isinstance(connection, str) else connection.public_url

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Define Flask routes
@app.route("/")
#This function called when user first enters url into browser
def home():
    return create_page(fpage, lgb='', knn='', logreg='', ann='', ensemble='', row_data='',
                          lgb_lime_table='',
                           logreg_lime_table = '',
                           knn_lime_table = '',
                           ann_lime_table = '')

@app.route('/data', methods = ['POST'])
#This function called when user hits Evaluate button
def data():

  global feature_names, lime_explainer, lgb_model, knn_model, logreg_model, ann_model, fitted_transformer

  form_data = request.form

  #get predictions
  new_row, yhat_lgb, yhat_knn, yhat_logreg, yhat_ann = handle_data(form_data.to_dict(), fitted_transformer, config, feature_names)  #calling my own function here
  ensemble = (yhat_lgb[0]+yhat_knn[0]+yhat_logreg[0]+yhat_ann[0])/4.0
  lgb = np.round(yhat_lgb[0], 2)
  knn = np.round(yhat_knn[0], 2)
  logreg = np.round(yhat_logreg[0], 2)
  ann = np.round(yhat_ann[0], 2)
  ensemble = np.round(ensemble, 2)

  #handle lime stuff
  if lime_explainer:
    try:
      logreg_explanation = lime_explainer.explain_instance(new_row[0], logreg_model.predict_proba, num_features=len(feature_names))
      lime_df = create_lime_table(logreg_explanation)
      logreg_lime_table = lime_df.to_html(index=False, justify='center').replace('<td>','<td style="text-align: center;">')
    except Exception as e:
      logreg_lime_table = e
      pass
    try:
      lgb_explanation = lime_explainer.explain_instance(new_row[0], lgb_model.predict_proba, num_features=len(feature_names))
      lime_df = create_lime_table(lgb_explanation)
      lgb_lime_table = lime_df.to_html(index=False, justify='center').replace('<td>', '<td style="text-align: center;">')
    except Exception as e:
      lgb_lime_table = e
      pass
    try:
      knn_explanation = lime_explainer.explain_instance(new_row[0], knn_model.predict_proba, num_features=len(feature_names))
      lime_df = create_lime_table(knn_explanation)
      knn_lime_table = lime_df.to_html(index=False, justify='center').replace('<td>', '<td style="text-align: center;">')
    except Exception as e:
      knn_lime_table = e
      pass
    try:
      ann_explanation = lime_explainer.explain_instance(new_row[0], ann_proba, num_features=len(feature_names))
      lime_df = create_lime_table(ann_explanation)
      ann_lime_table = lime_df.to_html(index=False, justify='center').replace('<td>', '<td style="text-align: center;">')
    except Exception as e:
      ann_lime_table = e
      pass

  #fill in fpage with results from models and Lime
  return create_page(fpage, lgb=lgb, knn=knn, logreg=logreg, ann=ann, ensemble=ensemble, row_data=str(form_data.to_dict()),
                           lgb_lime_table=lgb_lime_table,
                           logreg_lime_table = logreg_lime_table,
                           knn_lime_table = knn_lime_table,
                           ann_lime_table = ann_lime_table
                           )


# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


Connection: NgrokTunnel: "https://fcec-34-75-89-92.ngrok-free.app" -> "http://localhost:5000"


### You should see something like this

<pre>
Connection: NgrokTunnel: "https://a93f-35-237-39-159.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on
 </pre>

 The first url is the one that gets you to your page (and that you would hand out for others to see, as long as the server is running).

### The server is running in background at this point

In [36]:
foobar  #causes error but now you can go back and edit your code if you wish - the server is still running in another thread

 * Serving Flask app 'library'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


NameError: name 'foobar' is not defined